Ayudantía Martes 28 de agosto de 2019

# Actividad Web Scraping

## Indicaciones generales :

### • En esta actividad deberan ocupar Web Scraping para obtener una base de datos a partir de los datos de bolsos en https://www.newchic.com/es/.
### •  Deben obtener los datos de 100 productos con los atributos de: 
    •  Nombre
    •  Rating
    •  Precio
    •  Oferta
    •  Puntos
    •  Color
    •  Envío
    •  Especificación
    •  Comentarios (obtener primeros 5)

### •  Para esta actividad se debe entregar Notebook y archivo CSV con la base de datos.
### • Entrega martes 27 de agosto a las 10:59 pm.


## Elementos a usar para desarrollar esta actividad :

• Usar find(), find all() y select(). Todos permiten la selección de tags. A continuación se muestra un ejemplo de estructura html:
    - Tipo de tag
        Ej. <a>, <div ...>
    - Valores de atributos
        Ej. <div class=”abclinksmain”>)
    - N-ésimo hijo de/ Hermano de... <div class=”abclinksmain”>
        <ul>
        <li>—–Primer hijo de <ul> ...
        </li>
        <li>—–Segundo hijo de <ul>
        <a href=”http://www. ...”>
        A </a>
        </li>
        <li>—–Tercer hijo de <ul> ...
        </li>
        </u> </div>
    
• Las funciones find() y find all() reciben como parámetro filtros para seleccionar tags. Por otra parte, select() acepta selectores CSS para el filtro. El modo de uso de ambos se indica en la documentación (*).
    - Detalles de BeautifulSoup:
         Para extraer el valor de un atributo de un tag deben usar tag[“atributo”].
         Para acceder a su texto interior deben usar tag.text.
    
• Obtener página de Inicio : Lo primero que deben hacer es obtener la data html de la página principal haciendo uso de la librería requests. Posteriormente, deben procesar su data a un objeto BeautifulSoup para poder navegar y buscar fácilmente a través del árbol.
       - result = requests.get(path)
       - content = result.content
       - soup = BeautifulSoup(content, 'html.parser')

• Obtener url directorios : Si bien en el lapso de tiempo de esta clase no se descargará toda la base de datos, es necesario poder hacer una navegación profunda del sitio.
    
• Manejo de datos : Poner atención en el manejo de errores en casos en que algún atributo no exista. De no existir, rellenar con string o lista vacía ese atributo en la base de datos.
        
### (*) Para entender detalles de el uso de BeautifulSoup consultar la documentación https://www.crummy.com/software/BeautifulSoup/bs4/doc/


## 1. Librerias

In [50]:
import requests
from bs4 import BeautifulSoup, Tag
import pandas as pd
import numpy as np

## 2. Función para obtener los path de cada producto.

In [51]:
main_url = "https://es.newchic.com/bags-c-3579/"
#salida = open("base_bolso.csv","w",encoding="utf-8")
productos = []
lista = []
i = 0
cond = True
while cond:
    pageContent=requests.get(main_url+str(i)+".html")
    pageContent.encoding = 'utf-8'
    soup=BeautifulSoup(pageContent.text,"lxml")
    data = soup.find("ul", {"class": "clearfixes newlst_box product-list-js"}).find_all("li", {"class": "wom_lst_detail product-item-js"})
    for elemento in data:
        diccionario = {}
        nombre = elemento.find("div",{"class":"wom_lst_pic product-image-js"}).find("a", href=True)
        productos.append(nombre['href'])
        if len(productos)==100:
            cond = False
            break
        #precio = elemento.find("span",{"class":"price product-price-js"}).text
        #rating = ""
        #oferta = elemento.find("span",{"class":"product-discount-js"}).text
        #puntos = ""
        #color = ""
        #envio = " "
    i+=1
print(len(productos))
        
        #descr = elemento.find('a', href=True)
        #link = descr['href'].split("/")

#datos=pd.DataFrame(lista)
#datos.to_csv(salida,sep=';',index=False,encoding='utf-8')


100


In [52]:
productos[0]

'https://es.newchic.com/brenice-luggageandtravel-bags-4986/p-1509242.html?rmmds=category'

# 3.Funciones para extraer información del producto con los atributos que se señalan.

In [53]:
def get_text_with_br(tag, result=''):
    for x in tag.contents:
        if isinstance(x, Tag):  # check if content is a tag
            if x.name == 'br':  # if tag is <br> append it as string
                result += str("*")
            else:  # for any other tag, recurse
                result = get_text_with_br(x, result)
        else:  # if content is NavigableString (string), append
            result += x

    return result

#función para separar el texto dentro de in tag si tiene <br>, codigo usado de https://stackoverflow.com/questions/49649090/separating-by-br-tags-in-get-text

In [54]:
for path in productos:
    dic = {}
    pageContent=requests.get(path)
    pageContent.encoding = 'utf-8'
    soup=BeautifulSoup(pageContent.text,"lxml")
    data = soup.find("div", {"class": "container product-view-container"})
    nombre = data.find("h1", {"class": "d-inline product-info-product-name-js"}).text
    precio = data.find("span",{"class":"px-lg-2 price-number" }).text
    rating = data.find("div",{"class" :"flex align-items-center mx-lg-n2 mt-lg-5"}).find("span").text[0:3]
    oferta = data.find("span",{"class": "px-lg-5 bg-primary text-white font-small product-price__discount product-price-discount-js"}).text
    # puntos = data.find("ul",{"class": "product-price-activity-list-js"}) no los pude conseeguir, text no me dejaba acceder a ellos.
    envio = data.find("div",{"class": "product-info-ship-msg-js" }).text
    es_content = data.find("div",{"class" : "product-description-content font-normal mb-lg-20" })
    color = es_content.find("span")
    text = get_text_with_br(color).split("*")
    colores = ""
    especificacion = ""
    comments = ""
    if text[0] == 'especificación :':
        #borra el tag especificacion de la lista hecha anteriormente
        del(text[0])
        
    for elemento in text:
        if "color" in elemento.lower():
            #busco si en las especificaciones se menciona el color si lo menciona lo agrega, si no, lo deja en blanco
            if len(elemento.lower().split("color: "))>1:
                colores = elemento.lower().split("color: ")[1]
            else: 
                colores = elemento.lower().split("color: ")[0]
        elif elemento == "":
            continue
        else:
            #si no es color, lo agrego al string de especificaciones, no hace falta repetir el color creo.
            especificacion += elemento+","
    comentarios = data.find("div", {"class" :"reviews-list mb-lg-50"})
    if comentarios:
        comentarios = comentarios.find_all("div", {"class": "mb-lg-20"})
        k = 0
        for comentario in comentarios:
            
            com =  comentario.text.replace("\n","").strip()
            if com != "":
                comments +=com
                k+=1
                if (k == 5 or (k == len(comentarios))):
                    break
                comments += "-"
           
    #print(especificacion)
    dic["nombre"] = nombre
    dic["precio"] = precio
    dic["rating"] = rating
    dic["oferta"] = oferta
    dic["envio"] = envio
    dic["color"] = colores
    dic["especificacion"] = especificacion.replace("\n",",")
    dic["comentarios"] = comments
    lista.append(dic)
            

## 4. Cargar base de datos en un dataframe y almacenar en un archivo con extensión CSV.

In [55]:
datos=pd.DataFrame(lista)
#los comentarios estan separados por un -
datos.to_csv("out.csv",sep=';',index=False,encoding='utf-8')

In [56]:
print(len(lista))

100
